In [4]:
from __future__ import division, print_function
import numpy as np
import os
import sys
sys.path.append('../hw5/homework5_code')
from hmmlearn.hmm import MultinomialHMM

In [281]:
%run 'preprocessing.ipynb'

In [282]:
def get_rhymes(words):
    num_words = len(words)
    rhymes = np.empty((num_words, num_words))
    
    for i in range(num_words):
        for j in (k for k in range(num_words) if not k == i):
            if poetrytools.rhymes(words[i], words[j]):
                rhymes[i][j] = 1
    return rhymes

In [283]:
quatrain_rhymes = get_rhymes(quatrain_words)
volta_rhymes = get_rhymes(volta_words)
couplet_rhymes = get_rhymes(couplet_words)

In [284]:
reversed_quatrain_model = MultinomialHMM(n_components=20)
reversed_volta_model = MultinomialHMM(n_components=20)
reversed_couplet_model = MultinomialHMM(n_components=20)
reversed_quatrain_model.fit(np.concatenate(reversed_converted_quatrain_lines), quatrain_lengths)
reversed_volta_model.fit(np.concatenate(reversed_converted_volta_lines), volta_lengths)
reversed_couplet_model.fit(np.concatenate(reversed_converted_couplet_lines), couplet_lengths)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=20,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x10dabd640>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

In [327]:
def generate_b_line(seed, rhymes, model, length=7):
    b_line = []
    start_probs = model.startprob_
    emission_probs = model.emissionprob_
    transition_probs = model.transmat_

    start_state = np.random.choice(len(start_probs), p = start_probs)
    
    possible_start_emissions = np.where(rhymes[seed] == 1)
    probs = np.array(emission_probs[start_state][possible_start_emissions])

    scaled_probs = probs / sum(probs)
    
    start_emission = np.random.choice(possible_start_emissions[0], p=scaled_probs)
    b_line.append(start_emission)
    
    curr_state = start_state
    for _ in range(length - 1):
        possible_transitions = transition_probs[curr_state]
        curr_state = np.random.choice(len(possible_transitions), p=possible_transitions)
        possible_emissions = emission_probs[curr_state]
        curr_emission = np.random.choice(len(possible_emissions), p=possible_emissions)
        b_line.append(curr_emission)
        
    return b_line

def convert_a_line(sample, words):
    ret = ''
    reversed_list = []
    for word in np.nditer(sample):
        reversed_list.append(word)
    
    for word in reversed(reversed_list):
        ret += words[word] + ' '
    return ret

def convert_b_line(sample, words):
    ret = ''
    for word in reversed(sample):
        ret += words[word] + ' '
    return ret

def generate_pair(model, rhymes, length=7):
    while True:
        a_line = model.sample(length)[0]
        seed = a_line[0][0]
        print(np.where(rhymes[seed] == 1)[0])
        if len(np.where(rhymes[seed][0])) > 0:
            b_line = generate_b_line(seed, rhymes, model)
            return a_line, b_line
        
def generate_rhyming_sonnet():
    sonnet = ''
    a_lines = []
    b_lines = []
    
    for _ in range(4):
        a_line, b_line = generate_pair(reversed_quatrain_model, quatrain_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
        
    for i in range(2):
        sonnet += convert_a_line(a_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_a_line(a_lines[2 * i + 1], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i + 1], quatrain_words) + '\n'
    return sonnet

def generate_10_rhyming_sonnets():
    sonnets = ''
    for i in range(10):
        sonnets += str(i) + '\n' + generate_rhyming_sonnet() + '\n'
    
    f = open("project2data/rhyming_shakespeare.txt","w")
    f.write(sonnets)
    return sonnets

In [328]:
print(generate_rhyming_sonnet())

[ 667 1947]
[]


ValueError: a must be non-empty

In [270]:
a_lines

[array([[ 775],
        [1363],
        [ 803],
        [1335],
        [ 965],
        [ 105],
        [1228]])]

In [177]:
b_lines

[[262, 1226, 1668, 1310, 880, 1420, 390]]

In [158]:
quatrain_words[1238]

u'lasting'

In [159]:
quatrain_words[488]

u'Creating'